Chicago

In [1]:
# Importation des bibliothèques

import pandas as pd
import os
from rdflib import Graph

In [2]:
# la fonction de converti rdf vers dataframe
def rdf_daily_to_df(file_path):
    g = Graph()
    g.parse(file_path, format="xml")
    rows = {}

    for s, p, o in g:
        s, p, o = str(s), str(p), str(o)
        if s not in rows:
            rows[s] = {"subject": s, "city": "Chicago"}

        col = p.split("/")[-1]
        if col == "route": rows[s]["route"] = o
        elif col == "date": rows[s]["date"] = o
        elif col == "daytype": rows[s]["daytype"] = o
        elif col == "rides": rows[s]["ridership"] = o

    df = pd.DataFrame(rows.values())
    return df

In [3]:
# Lecture des fichiers rdf
dfs = []
input_folder = "../data_raw/rdf_CTA__Ridership__Daily_by_Route_routes_2001_2025/"

for file in os.listdir(input_folder):
    if file.endswith(".rdf"):
        full_path = os.path.join(input_folder, file)
        print(f"Reading: {file}")
        df_temp = rdf_daily_to_df(full_path)
        dfs.append(df_temp)

# Regroupement dans df_Daily
df_Daily = pd.concat(dfs, ignore_index=True)

Reading: rdf_CTA__Ridership__Daily_by_Route_routes_1.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_10.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_11.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_12.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_13.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_14.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_15.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_16.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_17.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_18.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_19.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_2.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_3.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_4.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_5.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_6.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_7.rdf
Reading: rdf_CTA__Rid

In [4]:
df_Daily.head()

,subject,city,route,ridership,date,daytype
0,https://data.cityofchicago.org/resource/jyb9-n...,Chicago,111,2198,2013-12-28T00:00:00,A
1,https://data.cityofchicago.org/resource/jyb9-n...,Chicago,1,2654,2005-10-13T00:00:00,W
2,https://data.cityofchicago.org/resource/jyb9-n...,Chicago,10,383,2010-11-30T00:00:00,W
3,https://data.cityofchicago.org/resource/jyb9-n...,Chicago,106,1486,2019-02-07T00:00:00,W
4,https://data.cityofchicago.org/resource/jyb9-n...,Chicago,11,501,2022-09-18T00:00:00,U


EDA

In [5]:
description = df_Daily.describe()
info = df_Daily.info()
taille = df_Daily.shape
les_null = df_Daily.isnull().sum()
Les_doublons = df_Daily.duplicated().sum()
print("Description de DataFrame \n", description)
print("Info sur la DataFrame Daily \n", info)
print("la taill de DataFrame \n", taille)
print("Les valeurs nulles dans le DataFrame  :\n", les_null)
print("Les doublons dans le DataFrame:", Les_doublons)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092474 entries, 0 to 1092473
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   subject    1092474 non-null  object
 1   city       1092474 non-null  object
 2   route      1092474 non-null  object
 3   ridership  1092474 non-null  object
 4   date       1092474 non-null  object
 5   daytype    1092474 non-null  object
dtypes: object(6)
memory usage: 50.0+ MB
Description de DataFrame 
                                                   subject     city    route  \
count                                             1092474  1092474  1092474   
unique                                            1092474        1      188   
top     https://data.cityofchicago.org/resource/jyb9-n...  Chicago      111   
freq                                                    1  1092474     9100   

       ridership                 date  daytype  
count    1092474              1092474  1092474 

Nettoyage

In [8]:
df_Daily["date"] = pd.to_datetime(df_Daily["date"], errors="coerce")
df_Daily["ridership"] = pd.to_numeric(df_Daily["ridership"], errors="coerce")

In [9]:
# Filtrage de la date
df_Daily=df_Daily[(df_Daily['date']> '2018-12-31')]
# supprimer colonnes subject et city des deux DataFrames
df_Daily = df_Daily.drop(columns=["subject", "city"])

In [10]:
# Exportation vers csv
df_Daily.to_csv("../data/chicago_rdf_combined.csv", index=False)
print("Fichier exporté avec succès !")

Fichier exporté avec succès !


Lecture de fichier Excel

In [13]:
# Chargement du fichier Excel de Chicago
path_excel = "../data_raw/cta-ridership-daily-boarding-totals-20260203-69820a3f9df63091665572.xlsx"
df_Excel = pd.read_excel(path_excel)

# Affichage des premières lignes pour vérifier
df_Excel.head()

,service_date,day_type,bus,rail_boardings,total_rides
0,2001-01-01,U,297192,126455,423647
1,2001-01-02,W,780827,501952,1282779
2,2001-01-03,W,824923,536432,1361355
3,2001-01-04,W,870021,550011,1420032
4,2001-01-05,W,890426,557917,1448343


In [12]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   --------------------

EDA de fichier excel

In [14]:
description = df_Excel.describe()
info = df_Excel.info()
taille = df_Excel.shape
les_null = df_Excel.isnull().sum()
Les_doublons = df_Excel.duplicated().sum()
print("Description de DataFrame \n", description)
print("Info de DataFrame  \n", info)
print("la taill de DataFrame \n", taille)
print("Les valeurs nulles dans le DataFrame  :\n", les_null)
print("Les doublons dans le DataFrame:", Les_doublons)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9100 entries, 0 to 9099
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   service_date    9100 non-null   datetime64[ns]
 1   day_type        9100 non-null   object        
 2   bus             9100 non-null   int64         
 3   rail_boardings  9100 non-null   int64         
 4   total_rides     9100 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 355.6+ KB
Description de DataFrame 
                         service_date           bus  rail_boardings  \
count                           9100  9.100000e+03    9.100000e+03   
mean   2013-06-16 12:01:25.846153984  7.044593e+05    5.064497e+05   
min              2001-01-01 00:00:00  8.078300e+04    2.354400e+04   
25%              2007-03-25 18:00:00  4.607805e+05    3.265512e+05   
50%              2013-06-16 12:00:00  7.505730e+05    5.718135e+05   
75%         

Nettoyage de excel

In [ ]:
# Conversion de la colonne date en format datetime
df_Excel['service_date'] = pd.to_datetime(df_Excel['service_date'])

# Filtrage pour garder uniquement les données après 2018
df_Excel = df_Excel[(df_Excel['service_date'] > '2018-12-31')]

# Affichage des types de colonnes après nettoyage
df_Excel.info()

Exportation CSV

In [ ]:
# Exportation vers un fichier CSV final
df_Excel.to_csv("../data/chicago_excel_cleaned.csv", index=False)
print("Fichier Excel nettoyé et exporté avec succès !")

Fichier Excel nettoyé et exporté avec succès !


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Philadelphia